In [1]:
#installing ollama
#!pip install ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
#starting ollama server locally
import subprocess
import time
process = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)  # Wait for 5 seconds

In [3]:
#pulling llama3 using ollama
!ollama pull llama3.1:8b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling 667b0c1932bc...   0% ▕▏    0 B/4.9 GB                  pulli

In [4]:
#testing llama 3 is availble
!ollama list

NAME           ID              SIZE      MODIFIED               
llama3.1:8b    46e0c10c039e    4.9 GB    Less than a second ago    


In [26]:
!pip install langchain_community faiss-cpu eml_parser pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 23.2 MB/s eta 0:00:00


In [13]:
import os
import json
import torch
from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import faiss
import json
import hashlib
import numpy as np
import email
import os
import eml_parser
from datetime import datetime
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from scipy.spatial.distance import cosine
from bs4 import BeautifulSoup

In [28]:
# Load LLaMA 3 Model
llm = Ollama(model="llama3.1:8b")

In [15]:
# Define request types and subcategories
request_types = {
    "Adjustment": ["Reallocation Fees", "Amendment Fees", "Reallocation Principal"],
    "AU Transfer": [],
    "Closing Notice": ["Cashless Roll", "Decrease", "Increase"],
    "Commitment Change": ["Ongoing Fee", "Letter of Credit Fee"],
    "Fee Payment": ["Principal", "Interest", "Principal + Interest", "Principal + Interest + Fee"],
    "Money Movement - Inbound": ["Timebound"],
    "Money Movement - Outbound": ["Foreign Currency"]
}

In [16]:
# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [22]:

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_paths):
    """Loads PDF documents using PyPDFLoader and returns a list of documents."""
    documents = []
    for pdf_path in pdf_paths:
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
    text = ""
    for page in documents:
        text += page.page_content
    return text  # Load and extend the documents list

# Function to extract text from EML files
def extract_text_from_eml(eml_path):
    with open(eml_path, 'rb') as f:
        raw_email = f.read()
    parsed_eml = eml_parser.eml_parser.decode_email_b(raw_email)
    email_text = parsed_eml.get('body', [''])[0]
    if 'html' in parsed_eml:
        soup = BeautifulSoup(parsed_eml['html'][0], 'html.parser')
        email_text = soup.get_text()
    return email_text

In [40]:
import re
import json

def extract_json_from_string(text):
    match = re.search(r'\{(.*?)\}', text, re.DOTALL)
    if match:
        json_string = match.group(0)
        try:
            return json.loads(json_string)
        except json.JSONDecodeError:
            return None  # Handle invalid JSON
    return None  # No match found

In [44]:
# Function to classify email using LLaMA 3
import re
def classify_email(email_text):
    prompt = f"""
    Given the following email, classify it into a predefined request type and sub-request type.

    Email: "{email_text}"

    Available Request Types and Subtypes: {json.dumps(request_types, indent=2)}

    Respond with a JSON object like this:
    {{
        "request_type": "string",
        "sub_request_type": "string"
    }}
    """

    response = llm.invoke(prompt)
    print(response)
    classification = {"request_type": "Unknown", "sub_request_type": "Unknown"}
    try:

        extract_json_from_string(response)
        classification = extract_json_from_string(response)
        if classification:
            return classification
        else:
            print("No JSON object found in the response.")
    except json.JSONDecodeError:
        classification = {"request_type": "Unknown", "sub_request_type": "Unknown"}
    print("only json", classification)
    return classification

In [21]:
# Function to compute confidence score using embeddings
def compute_confidence(email_text, classification):
    email_vector = np.array(embedding_model.embed_query(email_text)).astype("float32")
    classification_text = classification["request_type"] + " " + classification["sub_request_type"]
    classification_vector = np.array(embedding_model.embed_query(classification_text)).astype("float32")
    similarity_score = 1 - cosine(email_vector, classification_vector)
    confidence_score = round(similarity_score * 100, 2)
    return confidence_score

# Function to check for duplicate emails
def is_duplicate(email_text, existing_hashes):
    email_hash = hashlib.md5(email_text.encode()).hexdigest()
    if email_hash in existing_hashes:
        return True, email_hash
    existing_hashes.add(email_hash)
    return False, email_hash

In [48]:
# Function to extract key data points dynamically
def extract_key_data(text):
    prompt = f"""
    Extract key data points from the following text:

    Text: "{text}"

    Identify fields such as sender name, amount, expiration date, and date to transfer, account number.
    Respond in JSON format like this:
    {{
        "sender_name": "string",
        "amount": "string",
        "expiration_date": "string",
        "date_to_transfer": "string"
    }}
    """

    response = llm.invoke(prompt)
    try:
        extracted_data = extract_json_from_string(response)
    except json.JSONDecodeError:
        extracted_data = {}
    return extracted_data

In [51]:
def generate_email_hash(email_text):
    """Generates an MD5 hash for the given email text."""
    return hashlib.md5(email_text.encode()).hexdigest()


In [52]:
# Process emails
existing_hashes = set()
email_files = ["loanServicingRequest.pdf"]  # Example files
results = []

for file in email_files:
    if file.endswith(".pdf"):
        email_text = extract_text_from_pdf([file])
    elif file.endswith(".eml"):
        email_text = extract_text_from_eml(file)
    else:
        continue
    print(email_text)

    extracted_data = extract_key_data(email_text)
    classification = classify_email(email_text)
    confidence = compute_confidence(email_text, classification)
    is_dup, email_hash = is_duplicate(email_text, existing_hashes)

    response_json = {
        "email_text": email_text[:100] + "...",  # Truncated for readability
        "classification": classification,
        "confidence_score": float(confidence),
        "duplicate": is_dup,
        "key_data": extracted_data,
        "email_hash": email_hash
    }
    results.append(response_json)

# Print results in JSON format
print(json.dumps(results, indent=2))


Loan Servicing Request
Date: 2025-04-05
From: finance@abccompany.com
To: loanservicing@bank.com
Subject: Adjustment Request - Reallocation Fees
Dear Loan Servicing Team,
We request an adjustment to the loan agreement for account #123456. Please reallocate the funds
and apply the reallocation fees.
Amount: $50,000  
Effective Date: 2025-04-10  
Best,  
Finance Team  
ABC Corp
Here is the classification of the email into a predefined request type and sub-request type:

```
{
  "request_type": "Adjustment",
  "sub_request_type": "Reallocation Fees"
}
```

Explanation: The email subject mentions "Adjustment Request" which corresponds to the "Adjustment" request type. Within that, it specifically requests the reallocation of fees for a loan agreement, which matches the "Reallocation Fees" sub-request type.
[
  {
    "email_text": "Loan Servicing Request\nDate: 2025-04-05\nFrom: finance@abccompany.com\nTo: loanservicing@bank.com\nSubj...",
    "classification": {
      "request_type": "Adjus